# setup langsmith 

In [1]:
from uuid import uuid4
import os

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # Update to your API key

In [2]:
from langsmith import Client
client = Client()

# Load the database


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
import cassio
# connection of Astra DB
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

cassio.init(
    token = ASTRA_DB_APPLICATION_TOKEN,
    database_id = ASTRA_DB_ID
)

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

d:\TAMU\SEM 4\CNM\BrokenBasket\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Cassandra
astra_vector_store = Cassandra(embedding=embeddings,
                               table_name = "CNM_test_table",
                               session=None,
                               keyspace=None)

In [7]:
retriever = astra_vector_store.as_retriever()
retriever.invoke("Give me a cookie recipe")

[Document(id='53874fa27eca451281e0084ab85c34f3', metadata={'description': 'These are THE BEST soft chocolate chip cookies! No chilling required. Just ultra thick, soft, classic chocolate chip cookies!', 'language': 'en-US', 'source': 'https://pinchofyum.com/the-best-soft-chocolate-chip-cookies', 'title': 'The Best Soft Chocolate Chip Cookies Recipe - Pinch of Yum'}, page_content='Me \nLindsay was kind enough to post this perfect recipe. I think you can figure this one out yourself‚Ä¶ Smh\n\n\n\n04/21/16 @ 8:56 pm\n\n\nReply \n\n\n\n\n\n\n \n\n\n\nhoi \nYah, toats agree vith Zou!\n \n\n04/30/16 @ 10:00 am\n\n\n\n\n\n\n\n\n\n \n\n\n\nKevin \nMaybe you shouldn‚Äôt be so weirdly hateful in the comments section of a yummy chocolate chip cookies recipe? This is supposed to be fun, right?\n\n\n\n12/24/16 @ 9:30 pm\n\n\n\n\n\n\n\n\n\n \n\n\n\njacquelyn karney \nIf you are making chocolate chip cookies, who cares about the nutritional information?\n \n\n06/15/19 @ 11:36 pm\n\n\n\n\n\n\n\n\n\n \

# Langgraph 

In [8]:
# langgraph application
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser


d:\TAMU\SEM 4\CNM\BrokenBasket\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
# Data model
class RouteQuery(BaseModel):
  """Route a user query to the most relevant datasource"""
  datasource: Literal["vectorstore","chat"] = Field(
      ...,
      description="Given a user question choose to route it to chat or a vectorstore"
  )

In [10]:
from langchain_groq import ChatGroq
import os
groq_api_key = os.getenv('groq_api_key')
# print(groq_api_key)

In [11]:
llm=ChatGroq(groq_api_key=groq_api_key,model_name='Llama-3.3-70b-Versatile')
structured_llm_router=llm.with_structured_output(RouteQuery)

In [36]:
# Prompt
system = """You are an expert at routing a user question to a vectorstore or chat.
The vectorstore contains documents related to baking recipes.
Use the vectorstore for questions on these topics. Otherwise, use chat."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{message}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"message": "what is stardew Valley"}
    )
)
print(question_router.invoke({"message": "How to make a sweet dessert"}))

datasource='chat'
datasource='vectorstore'


In [37]:
chat_llm = ChatGroq(groq_api_key=groq_api_key,model_name='Llama-3.3-70b-Versatile')

In [38]:
## Graph

from typing import List,Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph import add_messages
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    messages: Annotated[list[AnyMessage], add_messages]
    documents: List[str]

In [ ]:
from langchain_core.messages import trim_messages

def trim(state):
    response = trim_messages(
        state["messages"],
        # Keep the last <= n_count tokens of the messages.
        strategy="last",
        token_counter=len,
        # When token_counter=len, each message
        # will be counted as a single token.
        # Remember to adjust for your use case
        max_tokens=4,
        # Most chat models expect that chat history starts with either:
        # (1) a HumanMessage or
        # (2) a SystemMessage followed by a HumanMessage
        start_on="human",
        # Most chat models expect that chat history ends with either:
        # (1) a HumanMessage or
        # (2) a ToolMessage
        end_on=("human", "tool"),
        # Usually, we want to keep the SystemMessage
        # if it's present in the original history.
        # The SystemMessage has special instructions for the model.
        include_system=True,
    )
    return {"messages": response}

In [57]:
def chat_bot(state):
    response = chat_llm.invoke(state["messages"])
    return {"messages": response}

In [58]:

from langgraph.graph import END, START, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage


workflow = StateGraph(GraphState)

workflow.add_node("chat", chat_bot)
workflow.add_node("trim", trim)

workflow.add_edge(START, "trim")
workflow.add_edge("trim", "chat")
workflow.add_edge("chat", END)

memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [59]:
config = {"configurable": {"thread_id": "1"}}

input_message = [HumanMessage(content="What is my name?")]
output = app.invoke({"messages":input_message},config=config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. I'm a large language model, I don't have the ability to know personal information about you, including your name. I can only respond based on the input you provide. If you'd like to share your name, I'd be happy to chat with you and use it in our conversation.


In [60]:
input_message = [HumanMessage(content="my name is John")]
output = app.invoke({"messages":input_message},config=config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, John. It's great that you've shared your name with me. I'll do my best to address you by name in our conversation. Is there something I can help you with or would you like to chat about a particular topic, John?


In [61]:
input_message = [HumanMessage(content="is Aarush dumb?")]
output = app.invoke({"messages":input_message},config=config)
